In [22]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rkuo2000/uecfood256")

print("Path to dataset files:", path)

100%|██████████| 3.94G/3.94G [00:46<00:00, 91.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rkuo2000/uecfood256/versions/1


In [24]:
import os

dataset_path = "/content/uecfood256"

# Check if the directory exists
if os.path.exists(dataset_path):
    print("✅ Directory exists:", dataset_path)
    print("📂 Files:", os.listdir(dataset_path))
else:
    print("❌ Directory not found!")

✅ Directory exists: /content/uecfood256
📂 Files: ['UECFOOD256', '1']


In [25]:
import os

dataset_path = "/content/uecfood256/UECFOOD256"
print("📂 Files in UECFOOD256:", os.listdir(dataset_path))

📂 Files in UECFOOD256: ['62', '248', '218', '255', '14', '225', '148', '183', '44', '74', '231', '81', '115', '171', '247', '17', '213', '138', '169', '142', '88', '27', '112', '66', '233', '56', '11', '84', '25', '217', '132', '140', '178', '214', '194', '220', '207', '216', '72', '40', '99', '134', '116', '246', '154', '73', '252', '245', '172', '121', '201', '187', '23', '98', '2', '166', '170', '147', '90', '230', '143', '52', '181', '137', '202', '5', '3', '128', '110', '253', '58', '163', '222', '168', '111', '7', '173', '156', '224', '6', '145', '242', '68', '113', '48', '199', '186', '109', '30', '175', '223', '118', '26', '176', '122', '254', '31', '209', '76', '24', '10', '125', '50', '241', '19', '182', '4', 'README.txt', '193', '102', '155', '235', '54', '184', '37', '249', '139', '9', '135', '195', '65', '18', '117', '119', '200', '1', '97', '188', '36', '211', '41', '146', '108', '161', '185', '151', '159', '20', '114', '234', '150', '55', '86', '256', '251', '15', '244',

In [29]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
data_dir = "/content/uecfood256/UECFOOD256"
train_dir = "food_dataset/train"
test_dir = "food_dataset/test"

# Create train/test folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get all food categories
food_classes = os.listdir(data_dir)

# Split and move images
for food in food_classes:
    # Check if it's a directory before proceeding
    if os.path.isdir(os.path.join(data_dir, food)):
        # Updated path to read images from the correct directory
        images = os.listdir(os.path.join(data_dir, food))

        # Check if the directory contains any images
        if len(images) > 0: # This check makes sure we have images to split
            train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

            os.makedirs(os.path.join(train_dir, food), exist_ok=True)
            os.makedirs(os.path.join(test_dir, food), exist_ok=True)

            for img in train_images:
                shutil.move(os.path.join(data_dir, food, img), os.path.join(train_dir, food, img))

            for img in test_images:
                shutil.move(os.path.join(data_dir, food, img), os.path.join(test_dir, food, img))
        else:
            print(f"Skipping directory '{food}' as it contains no images.")

print("✅ Dataset is ready with train/test split!")

Skipping directory '62' as it contains no images.
Skipping directory '248' as it contains no images.
Skipping directory '218' as it contains no images.
Skipping directory '255' as it contains no images.
Skipping directory '14' as it contains no images.
Skipping directory '225' as it contains no images.
Skipping directory '148' as it contains no images.
Skipping directory '183' as it contains no images.
Skipping directory '44' as it contains no images.
Skipping directory '74' as it contains no images.
Skipping directory '231' as it contains no images.
Skipping directory '81' as it contains no images.
Skipping directory '115' as it contains no images.
Skipping directory '171' as it contains no images.
Skipping directory '247' as it contains no images.
Skipping directory '17' as it contains no images.
Skipping directory '213' as it contains no images.
Skipping directory '138' as it contains no images.
Skipping directory '169' as it contains no images.
Skipping directory '142' as it contai

In [30]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ Loaded {len(train_dataset)} training images and {len(test_dataset)} test images.")

✅ Loaded 25011 training images and 6384 test images.


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet-50
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))  # Modify last layer
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Save trained model
torch.save(model.state_dict(), "food_model.pth")
print("✅ Model training completed and saved!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 136MB/s]


Epoch 1, Loss: 5.3183
Epoch 2, Loss: 4.7579
Epoch 3, Loss: 4.2387
Epoch 4, Loss: 3.7707
Epoch 5, Loss: 3.3377
✅ Model training completed and saved!


In [51]:
print(train_dataset.classes)

['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '

In [52]:
# Read category mapping file (UECFOOD256/category.txt)
def load_class_labels(file_path):
    class_mapping = {}
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                class_id = parts[0]  # Numeric ID as string
                class_name = " ".join(parts[1:])  # Join the rest as food name
                class_mapping[class_id] = class_name
    return class_mapping

# Load the class names from the dataset
category_file = "/content/uecfood256/UECFOOD256/category.txt"  # Adjust path as needed
class_labels = load_class_labels(category_file)
print(class_labels)  # Check the mapping

{'id': 'name', '1': 'rice', '2': 'eels on rice', '3': 'pilaf', '4': "chicken-'n'-egg on rice", '5': 'pork cutlet on rice', '6': 'beef curry', '7': 'sushi', '8': 'chicken rice', '9': 'fried rice', '10': 'tempura bowl', '11': 'bibimbap', '12': 'toast', '13': 'croissant', '14': 'roll bread', '15': 'raisin bread', '16': 'chip butty', '17': 'hamburger', '18': 'pizza', '19': 'sandwiches', '20': 'udon noodle', '21': 'tempura udon', '22': 'soba noodle', '23': 'ramen noodle', '24': 'beef noodle', '25': 'tensin noodle', '26': 'fried noodle', '27': 'spaghetti', '28': 'Japanese-style pancake', '29': 'takoyaki', '30': 'gratin', '31': 'sauteed vegetables', '32': 'croquette', '33': 'grilled eggplant', '34': 'sauteed spinach', '35': 'vegetable tempura', '36': 'miso soup', '37': 'potage', '38': 'sausage', '39': 'oden', '40': 'omelet', '41': 'ganmodoki', '42': 'jiaozi', '43': 'stew', '44': 'teriyaki grilled fish', '45': 'fried fish', '46': 'grilled salmon', '47': 'salmon meuniere', '48': 'sashimi', '49'

In [58]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Example: Manually assigned calorie information (You can expand this)
calorie_info = {
    "rice": 200,
    "miso soup": 80,
    "grilled fish": 250,
    "rolled egg": 150,
    "natto": 100,
    "eggplant miso soup": 90,
    "boiled spinach": 50,
    "tempura": 350,
    "croquette": 400,
    "yakitori": 250,
    "sushi": 200,
    "ramen": 450,
    "burger": 500,
    "pizza": 285,
    "pancakes": 350,
    "french fries": 365,
    "fried rice": 350,
    "donuts": 450,
    "hot dog": 290,
    "steak": 679,
    "salad": 150,
    "tacos": 250,
    "spaghetti bolognese": 380,
    "chicken wings": 450,
    "gyoza": 300,
    "curry rice": 400,
    "tonkatsu": 500,
    "udon": 350,
    "soba": 320,
    "stewed pork leg": 600,
    "grilled chicken": 300,
    "fish and chips": 700,
    "sandwich": 350,
    "fried chicken": 500,
    "dim sum": 280,
    "cheesecake": 450,
    "pudding": 220,
    "yakisoba": 400,
    "omelette rice": 450,
    "grilled eel": 450,
    "clam chowder": 250,
    "chicken curry": 550,
    "hamburger steak": 500,
    "bibimbap": 600,
    "sashimi": 150,
    "beef bowl": 700,
    "chow mein": 500,
    "hot pot": 450,
    "malasada": 350,
    "spring rolls": 200,
    "kimchi fried rice": 400,
    "pasta carbonara": 600,
    "lasagna": 700,
    "bruschetta": 170,
    "tiramisu": 450,
    "shawarma": 600,
    "falafel": 400,
    "kebab": 550,
    "hummus": 250
}

# Function to predict food from an image
def predict_food(image_path, model, class_labels):
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    # Load and preprocess image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)

    # Run model inference
    with torch.no_grad():
        output = model(image)
        predicted_class_id = torch.argmax(output, 1).item()

    # Convert class ID to actual food name
    predicted_class_id_str = str(predicted_class_id + 1)  # Ensure it's a string & adjust index
    predicted_food = class_labels.get(predicted_class_id_str, "Unknown")

    # Get calorie estimate
    estimated_calories = calorie_info.get(predicted_food.lower(), "Unknown")

    return predicted_food, estimated_calories

#Test the function with an image
image_path = "/content/uecfood256/1/UECFOOD256/10/10899.jpg"
predicted_food, estimated_calories = predict_food(image_path, model, class_labels)

#Display results
print(f"🍽️ Predicted Food: {predicted_food}")
print(f"🔥 Estimated Calories: {estimated_calories} kcal")


🍽️ Predicted Food: malasada
🔥 Estimated Calories: 350 kcal
